In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg

In [ ]:
i=0
YES_SOURCE_DIR = "facemask-dataset/data/with_mask/"
TRAINING_YES_DIR = "facemask-dataset/train-" + str(i+1) + "/with_mask/"
TESTING_YES_DIR = "facemask-dataset/test-" + str(i+1) + "/with_mask/"
NO_SOURCE_DIR = "facemask-dataset/data/without_mask/"
TRAINING_NO_DIR = "facemask-dataset/train-" + str(i+1) + "/without_mask/"
TESTING_NO_DIR = "facemask-dataset/test-" + str(i+1) + "/without_mask/"

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    dataset = []
    
    for unitData in os.listdir(SOURCE):
        data = SOURCE + unitData
        if(os.path.getsize(data) > 0):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file i.e zero size')
    
    train_set_length = int(len(dataset) * SPLIT_SIZE)
    test_set_length = int(len(dataset) - train_set_length)
    shuffled_set = random.sample(dataset, len(dataset))
    train_set = dataset[0:train_set_length]
    test_set = dataset[-test_set_length:]
       
    for unitData in train_set:
        temp_train_set = SOURCE + unitData
        final_train_set = TRAINING + unitData
        copyfile(temp_train_set, final_train_set)
    
    for unitData in test_set:
        temp_test_set = SOURCE + unitData
        final_test_set = TESTING + unitData
        copyfile(temp_test_set, final_test_set)

In [ ]:
def evaluate_model(TRAINING_YES_DIR, TESTING_YES_DIR, TRAINING_NO_DIR, TESTING_NO_DIR, i):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

    model.save_weights('model.h5')
    TRAINING_DIR = "facemask-dataset/train-" + str(i+1)
    train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

    train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
    VALIDATION_DIR = "facemask-dataset/test-" + str(i+1)
    validation_datagen = ImageDataGenerator(rescale=1.0/255)

    validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
    
    checkpoint_path = 'best_model-'+ str(i+1)
    mc = ModelCheckpoint(checkpoint_path,monitor='val_loss',verbose=0,save_best_only=True,mode='min')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)

    history = model.fit_generator(train_generator,
                              epochs = 2,
                              validation_data = validation_generator,
                              callbacks = [es, mc])

    model = load_model(checkpoint_path)

    test_acc = model.evaluate(validation_generator, verbose = 0)

    model.load_weights('model.h5')
    return model, test_acc

In [ ]:
n_folds = 5
cv_scores, model_history = list(), list()
for i in range(n_folds):
    YES_SOURCE_DIR = "facemask-dataset/data/with_mask/"
    TRAINING_YES_DIR = "facemask-dataset/train-" + str(i+1) + "/with_mask/"
    TESTING_YES_DIR = "facemask-dataset/test-" + str(i+1) + "/with_mask/"
    NO_SOURCE_DIR = "facemask-dataset/data/without_mask/"
    TRAINING_NO_DIR = "facemask-dataset/train-" + str(i+1) + "/without_mask/"
    TESTING_NO_DIR = "facemask-dataset/test-" + str(i+1) + "/without_mask/"
    # split data
    split_size = 0.8
    split_data(YES_SOURCE_DIR, TRAINING_YES_DIR, TESTING_YES_DIR, split_size)
    split_data(NO_SOURCE_DIR, TRAINING_NO_DIR, TESTING_NO_DIR, split_size)
    # evaluate model
    model, test_acc = evaluate_model(TRAINING_YES_DIR, TESTING_YES_DIR, TRAINING_NO_DIR, TESTING_NO_DIR, i)
    print("**********************************************************************************************")
    print('>%.3f' % test_acc[1])
    print("##############################################################################################")
    cv_scores.append(test_acc[1])
    model_history.append(model)
    
print('Estimated Accuracy %.3f (%.3f)' % (np.mean(cv_scores), np.std(cv_scores)))

In [ ]:
print(cv_scores)